In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, UpSampling2D, add
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import math

In [ ]:
# Filter the image data, low res image data/ High res image data from the dataset
Main_img_folder =  '../input/image-super-resolution-from-unsplash/Image Super Resolution - Unsplash'
low_res_img = os.path.join(Main_img_folder, 'low res')
high_res_img = os.path.join(Main_img_folder, 'high res') 

In [ ]:
# Read .csv file of image data
data = pd.read_csv("../input/image-super-resolution-from-unsplash/Image Super Resolution - Unsplash/image_data.csv")

#takes each element in the 'low_res' column of the 'data' DataFrame, assumes that these elements are filenames or paths, and joins them with the 'low_res_img' path to create complete file paths.
data['low_res']= [os.path.join(low_res_img, x) for x in data['low_res']]

#takes each element in the 'low_res' column of the 'data' DataFrame, assumes that these elements are filenames or paths, and joins them with the 'high_res_img' path to create complete file paths.
data['high_res']= [os.path.join(high_res_img, x) for x in data['high_res']]
data.head()

In [ ]:
# 4 mages will be loaded and processed together during each iteration of training or validation.
batch_size = 4 

In [ ]:
#resceld and split the train and validation data
datagen_low=ImageDataGenerator(rescale=1./255.,validation_split=0.25)
datagen_high=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

# create a genearator for the given dataframe for low & High resolution train and validation data for High and Low resolution
low_res_generator_train=datagen_low.flow_from_dataframe(
        data,
        x_col='low_res',
        target_size=(800, 1200),
        class_mode = None,
        batch_size = batch_size,
        seed=42,
        subset='training')

high_res_generator_train=datagen_high.flow_from_dataframe(
        data,
        x_col='high_res',
        target_size=(800, 1200),
        class_mode = None,
        batch_size = batch_size,
        seed=42,
        subset='training')

low_res_generator_val=datagen_low.flow_from_dataframe(
        data,
        x_col='low_res',
        target_size=(800, 1200),
        class_mode = None,
        batch_size = batch_size,
        seed=42,# means image loading and preprocessing process is reproducible.
        subset='validation')
high_res_generator_val=datagen_high.flow_from_dataframe(
        data,
        x_col='high_res',
        target_size=(800, 1200),
        class_mode = None,# dealing with image regression (predicting continuous values for high-resolution images), set class_mode to None, indicating that we are not using any class labels.
        batch_size = batch_size,
        seed=42,
        subset='validation')

In [ ]:
#Zipped all the train and test data into 2 different files
train_image_zip = zip(low_res_generator_train, high_res_generator_train)
val_image_zip = zip(low_res_generator_val, high_res_generator_val)

#This code will create a list of tuples containing (low_res, hi_res) pairs from the train_image_zip iterable
def imageGenerator(train_image_zip):
    for (low_res, hi_res) in train_image_zip:
            yield (low_res, hi_res)
    #return [(low_res, hi_res) for (low_res, hi_res) in train_image_zip]

In [ ]:

n = 0
#fig, axes = plt.subplots(1, 2, figsize=(20, 5))

for img, out in train_image_zip:
    if n < 5:
        fig, axes = plt.subplots(1, 2, figsize=(30, 10))
        axes[0].clear()
        axes[0].set_title('High Resolution Image', color='green', fontsize=20)
        axes[0].imshow(out[0])
        axes[0].axis('off')

        axes[1].clear()
        axes[1].set_title('Low Resolution Image', color='black', fontsize=20)
        axes[1].imshow(img[0])
        axes[1].axis('off')

        plt.pause(0.1)  # Pause briefly to allow the figure to update
        plt.show()

        n += 1
    else:
        break

In [ ]:
# input_img = tf.keras.layers.Input(shape=(800, 1200, 3))

# l1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(input_img)
# l2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(l1)
# l3 = tf.keras.layers.MaxPool2D(padding='same')(l2)

# l4 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(l3)
# l5 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(l4)
# l6 = tf.keras.layers.MaxPool2D(padding='same')(l5)

# l7 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(l6)

# l8 = tf.keras.layers.UpSampling2D()(l7)
# l9 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(l8)
# l10 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(l9)

# l11 = tf.keras.layers.add([l10, l5])

# l12 = tf.keras.layers.UpSampling2D()(l11)
# l13 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(l12)
# l14 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(l13)

# l15 = tf.keras.layers.add([l14, l2])

# decoded_image = tf.keras.layers.Conv2D(3, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu', activity_regularizer=tf.keras.regularizers.l1(10e-10))(l15)



**Autoencoders**
Autoencoders are an unsupervised learning technique in which we leverage neural networks for the task of representation learning. Specifically, we'll design a neural network architecture such that we impose a bottleneck in the network which forces a compressed knowledge representation of the original input.
As visualized below, we can take an unlabeled dataset and frame it as a supervised learning problem tasked with outputting x_hat, a reconstruction of the original input x.
Note: Here in our particular case we will try to reconstruct the low resolution images into their corresponding high resolution images.

https://towardsdatascience.com/intuitively-understanding-variational-autoencoders-1bfe67eb5daf

In [ ]:
input_img = tf.keras.layers.Input(shape=(800, 1200, 3)) 

l1 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu')(input_img)
l2 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu')(l1)
l3 = tf.keras.layers.MaxPool2D(padding='same')(l2)
l3 = Dropout(0.3)(l3)

l4 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu')(l3)
l5 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu')(l4)
l6 = tf.keras.layers.MaxPool2D(padding='same')(l5)

l7 = tf.keras.layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu')(l6)

l8 = tf.keras.layers.UpSampling2D()(l7)

l9 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu')(l8)
l10 = tf.keras.layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu')(l9)

l11 = tf.keras.layers.add([l10, l5])

l12 = tf.keras.layers.UpSampling2D()(l11)

l13 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu')(l12)
l14 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu')(l13)

l15 = tf.keras.layers.add([l14, l2])

decoded_image = tf.keras.layers.Conv2D(3, (3, 3), padding='same', kernel_initializer='he_uniform', activation='relu')(l15)



In [ ]:
model = tf.keras.models.Model(inputs=(input_img), 
                                     outputs=decoded_image)

model.compile( optimizer='Adam',  #'adadelta'
                     loss='mean_absolute_error', #lr = 0.01, loss="perceptual_loss",'mean_squared_error'/binary_crossentropy
                     metrics=['accuracy'])  

In [ ]:
model.summary()

In [ ]:
#count number of samples in train and val data
train_samples = high_res_generator_train.samples
val_samples = high_res_generator_val.samples

In [ ]:
#Preparing data for train and test to feed values in model.fit
train_data = imageGenerator(train_image_zip)
val_data = imageGenerator(val_image_zip)

In [ ]:
# Apply earlystopping, Modelcheckpoint and ReduceLRonplateau
early_stopper = EarlyStopping(monitor='val_loss', 
                              min_delta=0.01, 
                              patience=10, #50
                              verbose=1, 
                              mode='min')

model_checkpoint =  ModelCheckpoint(
                                    'model.h5', 
                                    save_best_only = True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.00000001)

https://saturncloud.io/blog/understanding-keras-resolving-the-mismatch-between-stepsperepoch-and-imagedatagenerator-output/#:~:text=In%20Keras%2C%20steps_per_epoch%20is%20a,divided%20by%20the%20batch%20size.


Use of Steps_per_epoch
https://datascience.stackexchange.com/questions/47405/what-to-set-in-steps-per-epoch-in-keras-fit-generator

In [ ]:
history = model.fit(
                        train_data,
                        steps_per_epoch=train_samples//batch_size,# math.ceil(train_samples//batch_size)
                        validation_data=val_data,
                        validation_steps=val_samples//batch_size,# math.ceil(val_samples//batch_size)
                        epochs=10, #500
                        callbacks=[early_stopper, model_checkpoint, learning_rate_reduction])

In [ ]:
# Curve of loss and Accuracy
plt.figure(figsize=(5,4))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()


plt.figure(figsize=(5,4))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
#Original Image, Ground Truth Image, Predicted Super Resolution Image
n = 0

for img, out in val_image_zip:
    pred = model.predict(img)
    if n < 5:
        fig, axes = plt.subplots(1, 3, figsize=(20, 4))

        axes[0].clear()
        axes[0].set_title('Low Resolution Image', fontsize=10)
        axes[0].imshow(img[0])
        axes[0].axis('off')
        
        
        axes[1].clear()
        axes[1].set_title('High Resolution Image', fontsize=10)
        axes[1].imshow(out[0])
        axes[1].axis('off')
        
        axes[2].clear()
        axes[2].set_title('Predicted High Resolution Image', fontsize=10)
        axes[2].imshow(pred[0])
        axes[2].axis('off')
        

        plt.pause(0.1)  # Pause briefly to allow the figure to update
        plt.show()

        n += 1
    else:
        break
        
    